## Initialize workspace

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [ ]:
import azureml
import random
from azureml.core import Workspace, Run
from azureml.core.authentication import MsiAuthentication
from azureml.core.workspace import Workspace

msiauth = MsiAuthentication()

ws = Workspace.from_config(auth=msiauth)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


## Deploy as web service
Once you've tested the model and are satisfied with the results, deploy the model as a web service hosted in ACI.
To build the correct environment for ACI, provide the following:
A scoring script to show how to use the model
An environment file to show what packages need to be installed
A configuration file to build the ACI
The model you trained before
### Create scoring script
Create the scoring script, called score.py, used by the web service call to show how to use the model.
You must include two required functions into the scoring script:
The init() function, which typically loads the model into a global object. This function is run only once when the Docker container is started.
The run(input_data) function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats are supported.


In [ ]:
%%writefile score.py
import numpy as np
import PIL
import json
from pandas import DataFrame
from keras.models import load_model
from azureml.core.model import Model
try: 
    from urllib.request import urlopen 
except ImportError: 
    from urllib import urlopen

def init():
    global model
    # retreive the path to the model file using the model name, this name should be from the last cell of the 02 notebook
    model_path = Model.get_model_path(<<name of the model>>)
    model = load_model(model_path)

def run(myimg):
    #myimg = np.array(PIL.Image.open(urlopen(url)), dtype=np.float32)
    data = np.array(json.loads(myimg)['data'])
    label_lookup = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]    
    image_data = np.array(data, dtype=np.uint8)
    image_data = image_data.astype('float32')    
    image_data = image_data.reshape(1,32, 32, 3)   
    result = np.squeeze(model.predict(image_data))
    json_d = {}
    # Return top 3 results:
    top_count = 3
    result_indices = (-np.array(result)).argsort()[:top_count]
    for i in range(top_count):
        json_d[label_lookup[result_indices[i]]]=(result[result_indices[i]] * 100)
    return json.dumps(json_d)

### Create environment file
Next, create an environment file, called myenv.yml, that specifies all of the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. This model needs `scikit-learn` and `azureml-sdk`.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("h5py==2.8.0")
myenv.add_pip_package("keras==2.1.5")
myenv.add_conda_package("numpy==1.15.1")
myenv.add_tensorflow_pip_package(core_type='cpu', version="1.10.1")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [ ]:
with open("myenv.yml","r") as f:
    print(f.read())

In [ ]:
# download the model 
from azureml.core import Workspace
from azureml.core.model import Model
import getpass

ws = Workspace.from_config(auth=msiauth)
## get the model details from modelname (need to replace)
model=Model(ws, getpass.getuser())

### Create configuration file
Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container. While it depends on your model, the default of 1 core and 1 gigabyte of RAM is usually sufficient for many models. If you feel you need more later, you would have to recreate the image and redeploy the service.

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "CIFAR10",  "method" : "tensorflow"}, 
                                               description='Predict CIFAR10 with tensorflow')

### Deploy in ACI
Estimated time to complete: about 7-8 minutes
Configure the image and deploy. The following code goes through these steps:
1.Build an image using:
    * The scoring file (score.py)
    * The environment file (myenv.yml)
    * The model file
Register that image under the workspace. 
Send the image to the ACI container.
Start up a container in ACI using the image.
Get the web service HTTP endpoint.

In [ ]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.image import Image

image_name=getpass.getuser()

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

# create the image
image = Image.create(name=image_name, models=[model], image_config=image_config, workspace=ws)

# wait for image creation to finish
image.wait_for_creation(show_output=True)


In [ ]:
##if the image failed or need to debug
print(image.image_build_log_uri)

In [ ]:
%%time
##Create the service


service_name=getpass.getuser()


# configure an ACI-based deployment
service = Webservice.deploy_from_image(workspace=ws,
                                       name=service_name,
                                       deployment_config=aciconfig,
                                       image=image)

service.wait_for_deployment(show_output=True)

service_name


In [ ]:
# if you already have the service object handy and want to see the logs
print(service.get_logs())


In [ ]:
##Get rest api
print(service.scoring_uri)

In [ ]:
from IPython.display import Image
import numpy as np
import PIL
try: 
    from urllib.request import urlopen 
except ImportError: 
    from urllib import urlopen
# Figure 6
Image(url="https://cntk.ai/jup/201/00014.png", width=64, height=64)


In [ ]:
import requests
import json
import numpy as np
from PIL import Image
try: 
    from urllib.request import urlopen 
except ImportError: 
    from urllib import urlopen

url = "https://cntk.ai/jup/201/00014.png"
myimg = np.array(PIL.Image.open(urlopen(url)), dtype=np.float32)


input_data=  "{\"data\": [" + str(myimg.tolist()) + "]}"


# for AKS deployment you'd need to the service key in the header as well
# api_key = service.get_key()
# headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)} 

headers = {'Content-Type':'application/json'}

resp = requests.post(service.scoring_uri, input_data, headers=headers)


print("Response", resp.text)


In [ ]:
service.delete()